In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import unidecode
import os

#### Funciones

In [14]:
def doClean(text):   
    
    # Quitar las expresiones tras la barra /
    a = re.sub('\/.*', '', text)
    # Quitar las aclaraciones entre paréntesis
    a = re.sub('\(.*\)', '', a)
    # Quitar acentos 
    a = unidecode.unidecode(a)
    # Quitar aquello que no sean palabras o cosa que se le parezca
    a = re.sub('\W', ' ', a) 
    # Quitar espacios extra en caso de haber
    a = re.sub('\s+', ' ', a)
    # Pasar el texto a minúsulas
    a = a.lower() 
    a = re.sub(' ', '_', a)
    #a = a.strip() 
    
    return a

#### Carga de ficheros

In [2]:
os.getcwd()

'/home/patcalsi/projects/Exodus_urbano/data/patri'

In [8]:
maestro_municipios = pd.read_csv("maestro_municipios.csv", index_col=0)

In [9]:
maestro_municipios.head()

,cp,municipio,provincia,ccaa
0,1001,vitoria_gasteiz,araba,pais_vasco
1,1002,vitoria_gasteiz,araba,pais_vasco
2,1003,vitoria_gasteiz,araba,pais_vasco
3,1004,vitoria_gasteiz,araba,pais_vasco
4,1005,vitoria_gasteiz,araba,pais_vasco


In [12]:
bibliotecas = pd.read_csv("sociedad_cultura_bibliotecas_2017.csv")

In [13]:
bibliotecas.head()

,municipio,total_bib
0,Adeje,5
1,Arafo,1
2,Arona,7
3,Barlovento,1
4,Breña Alta,1


In [15]:
bibliotecas['municipio_limpio'] = bibliotecas['municipio'].apply(doClean)

In [16]:
bibliotecas.head()

,municipio,total_bib,municipio_limpio
0,Adeje,5,adeje
1,Arafo,1,arafo
2,Arona,7,arona
3,Barlovento,1,barlovento
4,Breña Alta,1,brena_alta


In [23]:
hospitales = pd.read_csv("sociedad_sanidad_hospitales_2019.csv", sep='\t', encoding='latin1')

In [21]:
hospitales.head(50)

,CODCNH,NOMBRE,DIRECCION,TELEFONO,TELEFONO2,TELEFAX,CODMU,MUNICIPIOS,CODPROV,PROVINCIAS,...,HEM,ASD,LIT,BCO,ALI,SPECT,PET,MAMOS,DO,DIAL
0,10035,RED DE SALUD MENTAL DE ARABA (HOSPITAL PSIQUIµ...,"µLAVA, 43",945006555.0,NaN,945006587.0,10590,Vitoria-Gasteiz,1,µLAVA,...,0,0,0,0,0,0,0,0,0,0
1,10040,HOSPITAL SAN JOS,"BEATO TOMµS DE ZUMµRRAGA, 10",945140900.0,NaN,945145709.0,10590,Vitoria-Gasteiz,1,µLAVA,...,1,1,0,0,0,0,0,1,1,1
2,10053,HOSPITAL QUIRàNSALUD VITORIA,"ESPERANZA, 3",945252500.0,NaN,945279260.0,10590,Vitoria-Gasteiz,1,µLAVA,...,0,0,0,0,0,0,0,1,1,0
3,10066,"HOSPITAL DE CUIDADOS SAN ONOFRE, S.L. (HOSPITA...","SALVATIERRABIDE, 9",945142100.0,NaN,945143318.0,10590,Vitoria-Gasteiz,1,µLAVA,...,0,0,0,0,0,0,0,0,0,0
4,10088,HOSPITAL DE LEZA,"CTRA. VITORIA - LOGRO¥O, KM. 59",945006900.0,945006876.0,945006901.0,10318,Laguardia,1,µLAVA,...,0,0,0,0,0,0,0,0,0,0
5,10090,HOSPITAL UNIVERSITARIO DE ARABA (SEDE TXAGORRI...,"JOS ATXOTEGUI, S/N",945007000.0,945007007.0,945007359.0,10590,Vitoria-Gasteiz,1,µLAVA,...,2,3,1,0,2,1,1,3,1,57
6,20019,COMPLEJO HOSPITALARIO UNIVERSITARIO DE ALBACETE,"HERMANOS FALCà, 37",967597100.0,967597179.0,967243952.0,20030,Albacete,2,ALBACETE,...,1,1,1,0,2,0,0,2,1,16
7,20030,HOSPITAL GENERAL UNIVERSITARIO DE ALBACETE,"HERMANOS FALCà, 37",967597100.0,967597179.0,967243952.0,20030,Albacete,2,ALBACETE,...,0,0,0,0,0,0,0,0,0,0
8,20061,CLÖNICA SANTA CRISTINA ALBACETE,"PREZ GALDàS, 7",967223300.0,NaN,967507790.0,20030,Albacete,2,ALBACETE,...,0,0,0,0,0,0,0,0,0,0
9,20096,HOSPITAL DE HELLÖN,"JUAN RAMàN JIMNEZ, 40",967309500.0,967309517.0,967304611.0,20371,Hell¡n,2,ALBACETE,...,0,0,0,0,0,0,0,1,0,0


In [24]:
hospitales['municipio_limpio'] = hospitales['MUNICIPIOS'].apply(doClean)

In [26]:
hospitales.head()

,CODCNH,NOMBRE,DIRECCION,TELEFONO,TELEFONO2,TELEFAX,CODMU,MUNICIPIOS,CODPROV,PROVINCIAS,...,ASD,LIT,BCO,ALI,SPECT,PET,MAMOS,DO,DIAL,municipio_limpio
0,10035,RED DE SALUD MENTAL DE ARABA (HOSPITAL PSIQUIµ...,"µLAVA, 43",945006555.0,NaN,945006587.0,10590,Vitoria-Gasteiz,1,µLAVA,...,0,0,0,0,0,0,0,0,0,vitoria_gasteiz
1,10040,HOSPITAL SAN JOS,"BEATO TOMµS DE ZUMµRRAGA, 10",945140900.0,NaN,945145709.0,10590,Vitoria-Gasteiz,1,µLAVA,...,1,0,0,0,0,0,1,1,1,vitoria_gasteiz
2,10053,HOSPITAL QUIRàNSALUD VITORIA,"ESPERANZA, 3",945252500.0,NaN,945279260.0,10590,Vitoria-Gasteiz,1,µLAVA,...,0,0,0,0,0,0,1,1,0,vitoria_gasteiz
3,10066,"HOSPITAL DE CUIDADOS SAN ONOFRE, S.L. (HOSPITA...","SALVATIERRABIDE, 9",945142100.0,NaN,945143318.0,10590,Vitoria-Gasteiz,1,µLAVA,...,0,0,0,0,0,0,0,0,0,vitoria_gasteiz
4,10088,HOSPITAL DE LEZA,"CTRA. VITORIA - LOGRO¥O, KM. 59",945006900.0,945006876.0,945006901.0,10318,Laguardia,1,µLAVA,...,0,0,0,0,0,0,0,0,0,laguardia
